<a href="https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/hw4_statistical_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation with a Statistical Language Model
TODO: Explain what a statistical language model is 

**IMPORTANT:** The version of nltk that is available on colab is out-of-date. Make sure to use the following line to upgrade it, and then restart the Colab runtime before proceeding.

In [0]:
!pip install --upgrade nltk 

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=b1b8b1c811af5fe3a0d28854787291e8e08d97ca35ea6421fca773abacadce8e
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
import textwrap

import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
import nltk.lm.preprocessing 
from nltk.lm import MLE
from nltk.tokenize.treebank import TreebankWordDetokenizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Toy example of a statistical language model with NLTK
Below is a toy example of how to train a statistical language model with NLTK.

In [0]:

text = 'This is a sentence, and this is another sentence.'
tokenized_text = [
    list(map(str.lower, nltk.word_tokenize(sent)))
    for sent in nltk.sent_tokenize(text)
    ]

print('The training text is:' )
print(tokenized_text)

# Train a model that predicts the next word given the previous 2 words.
NGRAM_LENGTH = 3
print('{}-gram model'.format(NGRAM_LENGTH))
train_data, padded_sents = nltk.lm.preprocessing.padded_everygram_pipeline(NGRAM_LENGTH, tokenized_text)
lm = MLE(NGRAM_LENGTH)
lm.fit(train_data, padded_sents)

# There are 13 words total, of which two are "sentence"
print('P("sentence") = ', lm.score('a')) ## Expected output is 2/13

# "is" appears 2 times, and "is a" appear 1 time.
print('P("a"|"is") = ', lm.score('a', ['is'])) ## Expected output is 1/2

# "this is" appears 2 times, and "this is a" appear 1 time.
print('P("a"|"this is") = ', lm.score('a', 'this is'.split())) ## Expected output is 1/2

# Now train a model that predicts the next word given the previous 1 words.
NGRAM_LENGTH = 2
print('{}-gram model'.format(NGRAM_LENGTH))
train_data, padded_sents = nltk.lm.preprocessing.padded_everygram_pipeline(NGRAM_LENGTH, tokenized_text)
lm = MLE(NGRAM_LENGTH)
lm.fit(train_data, padded_sents)

# "is" appears 2 times, and "is a" appear 1 time.
print('P("a"|"is") = ', lm.score('a', ['is'])) ## Expected output is 1/2

# The 2-gram model only keeps track of the counts for text of length 2 words, so
# "this is a" is considered to have appeared 0 times.
print('P("a"|"this is") = ', lm.score('a', 'this is'.split())) ## Expected output is 0/2

## Generate presidential speeches with a staistical language model.
This section shows how we can use a statistical language model to generate text. For this example, you will be training the language model off of a corpus of presidential speeches.

#### Read in the data.

In [0]:
def process_text_corpus(path):
  with open(path) as f:
    text_lines = f.readlines()
  text = ' '.join(text_lines)
  tokenized_text = [
    list(map(str.lower, nltk.word_tokenize(sent)))
    for sent in nltk.sent_tokenize(text)
    ]
  return tokenized_text

# Download the train and test set.
!wget -nc -O /content/presidential_speeches_test.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
!wget -nc -O /content/presidential_speeches_train.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_train.txt

tokenized_text_train = process_text_corpus('/content/presidential_speeches_train.txt')
tokenized_text_test = process_text_corpus('/content/presidential_speeches_test.txt')
print(tokenized_text_test[:500])

File ‘/content/presidential_speeches_test.txt’ already there; not retrieving.
File ‘/content/presidential_speeches_train.txt’ already there; not retrieving.
[['<', 'title=', "''", 'address', 'to', 'the', 'general', 'assembly', 'of', 'the', 'united', 'nations', "''", '>', '<', 'president=', "''", 'ford', "''", '>', '<', 'date=', "''", 'september', '18', ',', '1974', "''", '>', 'mr.', 'president', ',', 'mr.', 'secretary', 'general', ',', 'your', 'excellencies', ':', 'in', '1946', ',', 'president', 'harry', 'truman', 'welcomed', 'representatives', 'of', '55', 'nations', 'to', 'the', 'first', 'general', 'assembly', 'of', 'the', 'united', 'nations', '.'], ['since', 'then', ',', 'every', 'american', 'president', 'has', 'had', 'the', 'great', 'honor', 'of', 'addressing', 'this', 'assembly', '.'], ['today', ',', 'with', 'pleasure', 'and', 'humility', ',', 'i', 'take', 'my', 'turn', 'in', 'welcoming', 'you', ',', 'the', 'distinguished', 'representatives', 'of', '138', 'nations', '.'], ['when', 

#### Build the model.
**TODO: talk about the tradeoffs between small NGRAM_LENGTH and large NGRAM_LENGTH.**

In [0]:
# You should experiment with varying this.
NGRAM_LENGTH = 3

print('{}-gram model'.format(NGRAM_LENGTH))
train_data, padded_sents = nltk.lm.preprocessing.padded_everygram_pipeline(NGRAM_LENGTH, tokenized_text_test)
lm = MLE(NGRAM_LENGTH)
lm.fit(train_data, padded_sents)

3-gram model


In [0]:
def detokenize(tokens):
  tokens_to_print = [t for t in tokens if t != '</s>' and t != '<s>']
  detokenizer = TreebankWordDetokenizer()
  return textwrap.wrap(detokenizer.detokenize(tokens_to_print))
  
# Change the random seed below to get a different generation.
print(detokenize(lm.generate(100, random_seed=42)))